# Calcul des vecteurs Word2Vec pour le BNC

In [1]:
import numpy as np
import gensim, logging
import operator
import scipy
import os
import xml.etree.ElementTree as et

In [2]:
def min_value (tupple_verb_occ, form):
    occ_inf = tupple_verb_occ[3]
    occ_p = tupple_verb_occ[4]
    occ_pp = tupple_verb_occ[5]
    min_occ_index = 0
    if form == "inf_p_pp":
        if occ_inf <= occ_p and occ_inf <= occ_pp:
            min_occ_index = 3
        if occ_p <= occ_inf and occ_p <= occ_pp:
            min_occ_index = 4
        if occ_pp <= occ_p and occ_pp <= occ_inf:
            min_occ_index = 5
    if form == "p_pp":
        if occ_p <= occ_pp:
            min_occ_index = 4
        if occ_pp <= occ_p:
            min_occ_index = 5
        return min_occ_index
    if form == "inf":
        min_occ_index = 3
    if form == "p":
        min_occ_index = 4
    if form == "pp":
        min_occ_index = 5

    return min_occ_index

In [3]:
def compute_cosine_similarity(list_predictor_positive, list_predictor_neg, homog_pos, nb_cosine_simi_sample):
    list_cosine_simi = []
    for e in range(len(list_predictor_positive)):
        predictor_pos = list_predictor_positive[e]
        predictor_neg = list_predictor_neg[e]
        cos_similiraty = new_model.most_similar(positive=[homog_pos, predictor_pos], negative=[predictor_neg], topn=nb_cosine_simi_sample)
        list_cosine_simi.append(cos_similiraty)

    return list_cosine_simi

In [4]:
# la fonction determine_nb_sample_cos_simi est inutile pour vous, elle m'a permis creuser des hypothèses d'optimisation de min côté 
def determine_nb_sample_cos_simi(upadate_simi_sample_double_if_true, value_sample_cos_similarity):
    nb_sample_similarity_updated = 0
    if upadate_simi_sample_double_if_true == "True":
        nb_sample_similarity_updated = value_sample_cos_similarity * 2
    else:
        nb_sample_similarity_updated = value_sample_cos_similarity

    return nb_sample_similarity_updated

In [5]:
def predict_form(cosine_similarities, len_list_predictors, nb_sample_cos_similarity, t_f_simi_sample_double):
    dict_cosine_similarities = {}
    result_prediction = 0
    nb_sample_similarity = determine_nb_sample_cos_simi(t_f_simi_sample_double, nb_sample_cos_similarity)
    for index in range(len_list_predictors):
        cosine_similarity_line = cosine_similarities[index]
        for i in range(nb_sample_similarity):
            one_cosine_similarity = cosine_similarity_line[i]
            str_verb_similarity = one_cosine_similarity[0]
            if str_verb_similarity in dict_cosine_similarities:
                dict_cosine_similarities[str_verb_similarity] = dict_cosine_similarities[str_verb_similarity] + 1
            else:
                dict_cosine_similarities[str_verb_similarity] = 1

    sorted_dic_simi = dict(sorted(dict_cosine_similarities.items(), key=operator.itemgetter(1), reverse=True))
    result_prediction = list(sorted_dic_simi.keys())[0]
    return result_prediction

In [6]:
def results(verb_to_predict_inf, dict_tupple_pred, pred_true_p, pred_true_pp):
    all_true_p = dict_tupple_pred[verb_to_predict_inf][0] + pred_true_p
    all_true_pp = dict_tupple_pred[verb_to_predict_inf][0] + pred_true_pp
    overall_success = all_true_p + all_true_pp
    pair_success = 0
    if pred_true_pp + pred_true_p == 2:
        pair_success = 1
    all_pair_success = dict_tupple_pred[verb_to_predict_inf][3] + pair_success
    updated_tupple = (all_true_p, all_true_pp, overall_success, all_pair_success)
    dict_tupple_pred[verb_to_predict_inf] = updated_tupple

In [7]:
def compute_accuracy_frequency(start_value, stop_value, step_value, dict_occ_tupple, dict_predict, form, print_true):
    for e in range(start_value, stop_value, step_value):
        print("______________________",e , " < Min occurence < ", e + step_value, "____________________________")
        sum_true_pred = 0
        nb_verbs_with_occ = 0
        for verb in dict_occ_tupple:
            index_min_occ = min_value(dict_occ_tupple[verb], form)
            if dict_occ_tupple[verb][index_min_occ] <= e + step_value:
                if dict_occ_tupple[verb][index_min_occ] > e :
                    sum_true_pred = sum_true_pred + dict_predict[verb][3]
                    nb_verbs_with_occ = nb_verbs_with_occ + 1
                    if print_true == "True":
                        print("Numbers of true pairs predicted for ", verb, ": ", dict_predict[verb][3])

        if nb_verbs_with_occ > 0:
            moy_true_pred = sum_true_pred / (nb_iter * nb_verbs_with_occ)
            print("Average true predicted pairs: ", moy_true_pred)

In [8]:
#___________________________________________________ hyper parameters __________________________________________________
# nb_iter = nombre d'itération de l'algorithme de prédiction
nb_iter = 40
# nombre de similarités cosinus retenues pour chaque verbe prédicteur qui serviront à la prédiction
nb_cosine_similarity_sample = 3

#hyper-parameters word2vec (see doc gensim). Ces variables modifient directement les paramètres du modéle word2vec (pas besoin d'aller fouiller dans le code)
min_frequency = 50
neurons = 200
window = 7
negative_sampling = 15
epochs = 10

In [9]:
#____________________________________________________ List verb samples __________________________________________________
#verbes dont on cherche à calculer les vecteurs de formes opposées (ils servent également de verbes prédicteurs)


list_all_verb_homog_inf = ["like", "apply", "emerge", "gain", "understand", "realize", "seek", "achieve", "approach", "love", "extend", "establish", "recognise", "miss", "acquire", "indicate", "point", "observe", "imagine", "attack", "attempt", "destroy", "stress", "accept", "drop", "marry", "argue", "earn", "demonstrate", "exist", "arrange", "collect", "prove", "note", "declare", "mention", "succeed", "discover", "deny", "form", "play", "return", "decide", "plan", "discuss", "launch", "inform", "disappear", "learn", "shoot", "claim", "arrive", "occur", "allow", "attend", "reject", "receive", "sign", "reach", "publish", "report", "present", "manage", "explain", "visit", "die", "advise", "repeat", "let", "seize", "lift", "check", "cross", "warn", "urge", "accuse", "save", "hope", "remind", "propose", "confirm", "promise", "expect", "teach", "replace", "regard", "ignore", "fight", "recommend", "issue", "sweep", "hang", "switch", "remove", "gather", "serve", "greet", "press", "lay", "prefer", "knock", "act", "force", "score", "back", "reduce", "prompt",
                    "develop", "feel", "bring", "show", "grow", "know", "see", "have_inf", "do_inf", "put", 'move', 'turn', 'set', 'tell', 'find', 'get', 'look', 'use',
                   'say', 'cut', 'hold', "make", "ask", "want", "call", "run", "need", "believe",
                   "happen", "remember", "produce", "help", "spend", "kill", "follow", "try", "carry",
                   "send", "notice", "push", "open", "win", "read", "fail", "involve", "describe", "add", "study",
                   "pick", "think", "close", "work", "live", "loose", "catch", "hear", "keep", "buy",
                   "assume", "provide", "consider", "express", "require", "fill", "mean", "enter", "face", "meet",
                   "hit", "finish", "pass", "raise", "lay", "join", "offer", "pay", "continue", "head", "settle", "order",
                   "recognize", "enjoy", "include", "agree", "cause"]
list_all_verb_homog_p = ["likedp", "appliedp", "emergedp", "gainedp", "understoodp", "realizedp", "soughtp", "achievedp", "approachedp", "lovedp", "extendedp", "establishedp", "recognisedp", "missedp", "acquiredp", "indicatedp", "pointedp", "observedp", "imaginedp", "attackedp", "attemptedp", "destroyedp", "stressedp", "acceptedp", "droppedp", "marriedp", "arguedp", "earnedp", "demonstratedp", "existedp", "arrangedp", "collectedp", "provedp", "notedp", "declaredp", "mentionedp", "succeededp", "discoveredp", "deniedp", "formedp", "playedp", "returnedp", "decidedp", "plannedp", "discussedp", "launchedp", "informedp", "disappearedp", "learnedp", "shotp", "claimedp", "arrivedp", "occurredp", "allowedp", "attendedp", "rejectedp", "receivedp", "signedp", "reachedp", "publishedp", "reportedp", "presentedp", "managedp", "explainedp", "visitedp", "diedp", "advisedp", "repeatedp", "letp", "seizedp", "liftedp", "checkedp", "crossedp", "warnedp", "urgedp", "accusedp", "savedp", "hopedp", "remindedp", "proposedp", "confirmedp", "promisedp", "expectedp", "taughtp", "replacedp", "regardedp", "ignoredp", "foughtp", "recommendedp", "issuedp", "sweptp", "hungp", "switchedp", "removedp", "gatheredp", "servedp", "greetedp", "pressedp", "laidp", "preferredp", "knockedp", "actedp", "forcedp", "scoredp", "backedp", "reducedp", "promptedp",
                    "developedp", "feltp", "broughtp", "showedp", "grewp", "knewp", "sawp", "hadp", "didp", "putp", 'movedp', 'turnedp', 'setp', 'toldp', 'foundp', 'gotp',
                     'lookedp', 'usedp', 'saidp', 'cutp', 'heldp', "madep", "askedp", "wantedp", "calledp",
                     "ranp", "neededp", "believedp", "happenedp", "rememberedp", "producedp",
                     "helpedp", "spentp", "killedp", "followedp", "triedp", "carriedp", "sentp",
                     "noticedp", "pushedp", "openedp", "wonp", "readp", "failedp", "involvedp", "describedp",
                     "addedp", "studiedp", "pickedp", "thoughtp", "closedp", "workedp", "livedp", "lostp",
                     "caughtp", "heardp", "keptp", "boughtp", "assumedp", "providedp",
                     "consideredp", "expressedp", "requiredp", "filledp", "meantp", "enteredp", "facedp", "metp",
                     "hitp", "finishedp", "passedp", "raisedp", "laidp", "joinedp", "offeredp", "paidp", "continuedp", "headedp",
                     "settledp", "orderedp", "recognizedp", "enjoyedp", "includedp", "agreedp", "causedp"]
list_all_verb_homog_pp = ["likedpp", "appliedpp", "emergedpp", "gainedpp", "understoodpp", "realizedpp", "soughtpp", "achievedpp", "approachedpp", "lovedpp", "extendedpp", "establishedpp", "recognisedpp", "missedpp", "acquiredpp", "indicatedpp", "pointedpp", "observedpp", "imaginedpp", "attackedpp", "attemptedpp", "destroyedpp", "stressedpp", "acceptedpp", "droppedpp", "marriedpp", "arguedpp", "earnedpp", "demonstratedpp", "existedpp", "arrangedpp", "collectedpp", "provedpp", "notedpp", "declaredpp", "mentionedpp", "succeededpp", "discoveredpp", "deniedpp", "formedpp", "playedpp", "returnedpp", "decidedpp", "plannedpp", "discussedpp", "launchedpp", "informedpp", "disappearedpp", "learnedpp", "shotpp", "claimedpp", "arrivedpp", "occurredpp", "allowedpp", "attendedpp", "rejectedpp", "receivedpp", "signedpp", "reachedpp", "publishedpp", "reportedpp", "presentedpp", "managedpp", "explainedpp", "visitedpp", "diedpp", "advisedpp", "repeatedpp", "letpp", "seizedpp", "liftedpp", "checkedpp", "crossedpp", "warnedpp", "urgedpp", "accusedpp", "savedpp", "hopedpp", "remindedpp", "proposedpp", "confirmedpp", "promisedpp", "expectedpp", "taughtpp", "replacedpp", "regardedpp", "ignoredpp", "foughtpp", "recommendedpp", "issuedpp", "sweptpp", "hungpp", "switchedpp", "removedpp", "gatheredpp", "servedpp", "greetedpp", "pressedpp", "laidpp", "preferredpp", "knockedpp", "actedpp", "forcedpp", "scoredpp", "backedpp", "reducedpp", "promptedpp",
                      "developedpp", "feltpp", "broughtpp", "shownpp", "grownpp", "knownpp", "seenpp", "hadpp", "donepp", "putpp", 'movedpp', 'turnedpp', 'setpp', 'toldpp', 'foundpp',
                      'gotpp', 'lookedpp', 'usedpp', 'saidpp', 'cutpp', 'heldpp', "madepp", "askedpp", "wantedpp",
                      "calledpp", "runpp", "neededpp", "believedpp", "happenedpp",
                      "rememberedpp", "producedpp", "helpedpp", "spentpp", "killedpp", "followedpp",
                      "triedpp", "carriedpp", "sentpp", "noticedpp", "pushedpp", "openedpp", "wonpp", "readpp",
                      "failedpp", "involvedpp", "describedpp", "addedpp", "studiedpp", "pickedpp", "thoughtpp",
                      "closedpp", "workedpp", "livedpp", "lostpp", "caughtpp", "heardpp", "keptpp",
                      "boughtpp", "assumedpp", "providedpp", "consideredpp", "expressedpp", "requiredpp",
                      "filledpp", "meantpp", "enteredpp", "facedpp", "metpp", "hitpp", "finishedpp", "passedpp",
                      "raisedpp", "laidpp", "joinedpp", "offeredpp", "paidpp", "continuedpp", "headedpp", "settledpp",
                      "orderedpp", "recognizedpp", "enjoyedpp", "includedpp", "agreedpp", "causedpp"]

In [10]:
#_____________________________ Parse xml files from BNC (don't forget to copy the root to the folder in variable "folder_directory") _________________________


list_words_in_sentence = []
list_sentences = []
unc_removed = 0
folder_directory = '/Users/gilles/Downloads/download/Texts/B'

for subdir, dirs, files in os.walk(folder_directory):
    for file in files:
        print(os.path.join(subdir, file))
        arbre = et.parse(os.path.join(subdir, file))
        root = arbre.getroot()

        for elem in arbre.iter():
            if elem.tag == 's':
                for w in elem.iter():
                    if w.tag == "w":
                        if w.attrib["pos"] == "UNC":
                            unc_removed += 1
                        else:
                            new_tupple = (w.text, w.attrib["c5"])
                            list_words_in_sentence.append(new_tupple)

                    elif w.tag == "c":
                        if w.text == "." or w.text == "?" or w.text == "!":
                            list_sentences.append(list_words_in_sentence)
                            list_words_in_sentence = []

print("UNC removed: " + str(unc_removed))

/Users/gilles/Downloads/download/Texts/B/BP/BP6.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPA.xml
/Users/gilles/Downloads/download/Texts/B/BP/BP7.xml
/Users/gilles/Downloads/download/Texts/B/BP/BP5.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPC.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPB.xml
/Users/gilles/Downloads/download/Texts/B/BP/BP4.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPF.xml
/Users/gilles/Downloads/download/Texts/B/BP/BP0.xml
/Users/gilles/Downloads/download/Texts/B/BP/BP1.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPG.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPE.xml
/Users/gilles/Downloads/download/Texts/B/BP/BP3.xml
/Users/gilles/Downloads/download/Texts/B/BP/BP2.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPD.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPH.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPJ.xml
/Users/gilles/Downloads/download/Texts/B/BP/BPK.xml
/Users/gilles/Downloads/download/Texts/B/BP/BP9.xml
/Users/gille

/Users/gilles/Downloads/download/Texts/B/BN/BNH.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7M.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7L.xml
/Users/gilles/Downloads/download/Texts/B/B7/B78.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7N.xml
/Users/gilles/Downloads/download/Texts/B/B7/B79.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7K.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7J.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7H.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7D.xml
/Users/gilles/Downloads/download/Texts/B/B7/B72.xml
/Users/gilles/Downloads/download/Texts/B/B7/B73.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7E.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7G.xml
/Users/gilles/Downloads/download/Texts/B/B7/B71.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7F.xml
/Users/gilles/Downloads/download/Texts/B/B7/B74.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7B.xml
/Users/gilles/Downloads/download/Texts/B/B7/B7C.xml
/Users/gille

In [11]:
#_________________________________ tokenization ________________________________________
#remarque: je ne discrimine pas les formes infinitives (car j'aurai besoin d'ajouter le suffixe _inf à tous les verbes 
#de list_all_verb_homog_inf et j'ai la flemme, j'ai déjà discriminé les noms avec le suffixe _n et les formes p et pp ce qui limite les homographies). 
#Si vous n'avez pas la flemme, go ahead, c'est juste 3 lignes de code à ajouter et une liste à mofifier

list_sents_tok = []
count_words = 0
print_add_word = 0

for sentence in range(len(list_sentences)):
    word_sentence = list_sentences[sentence]
    list_raw_word_tag = []

    for word_in_sent in range(len(word_sentence)):
        word_tag = word_sentence[word_in_sent]
        list_raw_word_tag.append(word_tag)
        list_word_tok = []

    for z in range(len(list_raw_word_tag)):

        word_tag_check = list_raw_word_tag[z]
        word_check = word_tag_check[0]
        word_check = word_check.split()
        word_check = word_check[0]
        tag_check = word_tag_check[1].lower()
        word_check = word_check.lower()

        if (word_check != ".") and (word_check != ",") and (word_check != "!") and (word_check != "?") and (
                word_check != ":") and (word_check != "``") and (word_check != "\'\'") and (
                word_check != ";") and (
                word_check != "-") and (word_check != "--") and (word_check != "(") and (word_check != ")"):

            if (len(word_tag_check) <= 1):
                test = test + 1

            else:
                count_words += 1
                if (tag_check == "vvd"):
                    word_p = word_check + "p"
                    list_word_tok.append(word_p)

                elif (tag_check == "vdb"): #tag_check == "vdi" or
                    do = word_tag_check[0]
                    do_inf = "do" + "_inf"
                    list_word_tok.append(do_inf)

                elif ((tag_check == "vdd")):
                    didp = "didp"
                    list_word_tok.append(didp)

                elif ((tag_check == "vdn")):
                    donepp = "donepp"
                    list_word_tok.append(donepp)

                elif (tag_check == "vhb"): #tag_check == "vhi" or
                    have = word_tag_check[0]
                    have_inf = "have" + "_inf"
                    list_word_tok.append(have_inf)

                elif ((tag_check == "vhd")):
                    hadp = "hadp"
                    list_word_tok.append(hadp)

                elif ((tag_check == "vhn")):
                    hadpp = "hadpp"
                    list_word_tok.append(hadpp)

                elif ((tag_check == "nn0") or (tag_check == "nn1") or (tag_check == "nn2")):
                    noun_n = word_tag_check[0] + "_n"
                    list_word_tok.append(noun_n)

                elif (tag_check == "vvn"):
                    word_pp = word_check + "pp"
                    list_word_tok.append(word_pp)

                elif (tag_check == "vvb"):
                    word_inf = word_check
                    list_word_tok.append(word_inf)

                else:
                    list_word_tok.append(word_check)

    print_add_word += 1
    if print_add_word == 100:
        print("\rTokenized Words added:" + str(count_words), end="")
        print_add_word = 0
    list_sents_tok.append(list_word_tok)

print("\nTokenization process completed. Tokenized words from corpus: " + str(count_words))
print("___________________________________")

Tokenized Words added:6543220
Tokenization process completed. Tokenized words from corpus: 6543487
___________________________________


In [12]:
#________________________________________ compute verb samples frequency ___________________________________________________
#le tupple que Gilles avait demandé qu'on aurait pu associer à une classe lexique pour appliquer des fonctions (ce que j'en ai compris en tout cas)
dict_homog_occ_tupple = {}
dict_homog_predict_methode2 = {}
dict_homog_predict_methode1 = {}
dict_homog_predict_methode3 = {}
dict_homog_predict_methode3bis = {}
verb_added_to_dict = 0
for i in range(len(list_all_verb_homog_inf)):
    verb_form_inf = list_all_verb_homog_inf[i]
    verb_form_p = list_all_verb_homog_p[i]
    verb_form_pp = list_all_verb_homog_pp[i]
    count_occ_inf = 0
    count_occ_p = 0
    count_occ_pp = 0
    for sent in list_sents_tok:
        for word in sent:
            if word == verb_form_inf:
                count_occ_inf = count_occ_inf + 1
            if word == verb_form_p:
                count_occ_p = count_occ_p + 1
            if word == verb_form_pp:
                count_occ_pp = count_occ_pp + 1
    tupple_verb = (verb_form_inf, verb_form_p, verb_form_pp, count_occ_inf, count_occ_p, count_occ_pp)
    tupple_pred = (0, 0, 0, 0)
    dict_homog_occ_tupple[verb_form_inf] = tupple_verb
    dict_homog_predict_methode1[verb_form_inf] = tupple_pred
    dict_homog_predict_methode2[verb_form_inf] = tupple_pred
    dict_homog_predict_methode3[verb_form_inf] = tupple_pred
    dict_homog_predict_methode3bis[verb_form_inf] = tupple_pred
    verb_added_to_dict += 1
    print("\rVerb added in dict: " + str(verb_added_to_dict), end="")

Verb added in dict: 192

In [13]:
#------------------ keep samples of verbs with frequency > min_ferquency for training and prediciton  -----------------
#Pour un corpus donné je ne garde dans la liste des verbes dont on va prédire les formes que ceux que ont une fréquence > n, n = min_frequency

list_verb_homog_inf = []
list_verb_homog_p = []
list_verb_homog_pp = []
list_predictors_inf = []
list_predictors_p = []
list_predictors_pp = []


for verb in list_all_verb_homog_inf:
    tupple_verb = dict_homog_occ_tupple[verb]
    if tupple_verb[3] > min_frequency and tupple_verb[4] > min_frequency and tupple_verb[5] > min_frequency:
        list_verb_homog_inf.append(tupple_verb[0])
        list_verb_homog_p.append(tupple_verb[1])
        list_verb_homog_pp.append(tupple_verb[2])
        list_predictors_inf.append(tupple_verb[0])
        list_predictors_p.append(tupple_verb[1])
        list_predictors_pp.append(tupple_verb[2])
        print("Verb added to list predicor / prediction: " + tupple_verb[0])

Verb added to list predicor / prediction: like
Verb added to list predicor / prediction: apply
Verb added to list predicor / prediction: emerge
Verb added to list predicor / prediction: gain
Verb added to list predicor / prediction: understand
Verb added to list predicor / prediction: seek
Verb added to list predicor / prediction: approach
Verb added to list predicor / prediction: love
Verb added to list predicor / prediction: establish
Verb added to list predicor / prediction: recognise
Verb added to list predicor / prediction: miss
Verb added to list predicor / prediction: acquire
Verb added to list predicor / prediction: indicate
Verb added to list predicor / prediction: point
Verb added to list predicor / prediction: observe
Verb added to list predicor / prediction: accept
Verb added to list predicor / prediction: drop
Verb added to list predicor / prediction: marry
Verb added to list predicor / prediction: argue
Verb added to list predicor / prediction: earn
Verb added to list pre

In [14]:
#_________________________________________________ training and predict form ________________________________________________________________________

list_acc_p_methode1 = []
list_acc_pp_methode1 = []
list_acc_p_methode2 = []
list_acc_pp_methode2 = []
list_results = []

In [15]:
for iter_w2v in range(nb_iter):

#_________________________________ Word2vec initialization and training __________________________________
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
     #train word2vec
    model = gensim.models.Word2Vec(list_sents_tok, min_count=min_frequency, size=neurons, iter=epochs, window=window, negative=negative_sampling)
    model.save("word2vec_model_100n_spw_homog2")
    # load and use Word2Vec for cosine similarities
    new_model = gensim.models.Word2Vec.load('word2vec_model_100n_spw_homog2')


#________________________________________ predictive algorithm _________________________________________
# ----------------------------- methode 1 ------------------------------------
#------------------------------ predict p ----------------------------------
    list_similarity_p = []
    for homographic_pos in list_verb_homog_inf:
        simi = compute_cosine_similarity(list_predictors_p, list_predictors_inf, homographic_pos, nb_cosine_similarity_sample)
        list_similarity_p.append(simi)


    count_true_pred_p_methode1 = 0
    list_true_prediction_p_m1 = []
    for index in range(len(list_similarity_p)):
        prediction_true = 0
        one_cosine_similarity = list_similarity_p[index]
        predicted_form = predict_form(one_cosine_similarity, 72, nb_cosine_similarity_sample, "")

        if predicted_form == list_verb_homog_p[index]:
            prediction_true = 1
            count_true_pred_p_methode1 = count_true_pred_p_methode1 + 1

        list_true_prediction_p_m1.append(prediction_true)

    #_______________________________________ predict pp _________________________________________
    list_similarity_pp = []
    for homographic_pos in list_verb_homog_inf:
        simi = compute_cosine_similarity(list_predictors_pp, list_predictors_inf, homographic_pos, nb_cosine_similarity_sample)
        list_similarity_pp.append(simi)


    count_true_pred_pp_methode1 = 0
    list_true_prediction_pp = []
    for index in range(len(list_similarity_pp)):
        prediction_true = 0
        one_cosine_similarity = list_similarity_pp[index]
        predicted_form = predict_form(one_cosine_similarity, 72, nb_cosine_similarity_sample, "")

        if predicted_form == list_verb_homog_pp[index]:
            prediction_true = 1
            count_true_pred_pp_methode1 = count_true_pred_pp_methode1 + 1

        list_true_prediction_pp.append(prediction_true)

#-------------------------------- Results methode 1 for this iteration -----------------------------------

    pairs_true_methode1 = 0
    for index in range(len(list_true_prediction_pp)):
        true_pred_p = list_true_prediction_p_m1[index]
        true_pred_pp = list_true_prediction_pp[index]
        if true_pred_p == 1 and true_pred_pp == 1:
            pairs_true_methode1 = pairs_true_methode1 + 1


    for index in range(len(list_verb_homog_inf)):
        verb_inf_to_predict = list_verb_homog_inf[index]
        pred_true_p = list_true_prediction_p_m1[index]
        pred_true_pp = list_true_prediction_pp[index]
        results(verb_inf_to_predict, dict_homog_predict_methode1, pred_true_p, pred_true_pp)


    number_pred_computed = len(list_true_prediction_pp)
    prediction_success_p_methode1 = count_true_pred_p_methode1
    prediction_success_pp_methode1 = count_true_pred_pp_methode1

    pred_acc_p_methode1 = prediction_success_p_methode1 / number_pred_computed * 100
    pred_acc_pp_methode1 = prediction_success_pp_methode1 / number_pred_computed * 100
    overall_acc_pred_methode1 = (prediction_success_p_methode1 + prediction_success_pp_methode1) / (2 * number_pred_computed) * 100
    Pairs_accuracy_methode1 = pairs_true_methode1 / number_pred_computed * 100

# ----------------------------- methode 2 ------------------------------------
#------------------------------ predict p ----------------------------------

    simi_matrix = np.zeros((len(list_verb_homog_inf), len(list_predictors_inf)))

    list_similarity_p = []
    for index_homographic_pos in range(len(list_verb_homog_pp)):
        homographic_pos = list_verb_homog_pp[index_homographic_pos]
        homographic_to_predict = list_verb_homog_p[index_homographic_pos]
        simi = compute_cosine_similarity(list_predictors_p, list_predictors_pp, homographic_pos, nb_cosine_similarity_sample)
        list_similarity_p.append(simi)

    count_true_pred_p_methode2 = 0
    list_true_prediction_p = []
    for index in range(len(list_similarity_p)):
        prediction_true = 0
        one_cosine_similarity = list_similarity_p[index]
        predicted_form = predict_form(one_cosine_similarity, 72, nb_cosine_similarity_sample, "")

        if predicted_form == list_verb_homog_p[index]:
            prediction_true = 1
            count_true_pred_p_methode2 = count_true_pred_p_methode2 + 1

        list_true_prediction_p.append(prediction_true)

    #_______________________________________ predict pp _________________________________________
    list_similarity_pp = []
    for homographic_pos in list_verb_homog_p:
        simi = compute_cosine_similarity(list_predictors_pp, list_predictors_p, homographic_pos, nb_cosine_similarity_sample)
        list_similarity_pp.append(simi)


    count_true_pred_pp_methode2 = 0
    list_true_prediction_pp = []
    for index in range(len(list_similarity_pp)):
        prediction_true = 0
        one_cosine_similarity = list_similarity_pp[index]
        predicted_form = predict_form(one_cosine_similarity, 72, nb_cosine_similarity_sample, "")


        if predicted_form == list_verb_homog_pp[index]:
            prediction_true = 1
            count_true_pred_pp_methode2 = count_true_pred_pp_methode2 + 1

        list_true_prediction_pp.append(prediction_true)

    #-------------------------------- Results methode 2 for this iteration -----------------------------------

    pairs_true_methode2 = 0
    for index in range(len(list_true_prediction_pp)):
        true_pred_p = list_true_prediction_p[index]
        true_pred_pp = list_true_prediction_pp[index]
        if true_pred_p == 1 and true_pred_pp == 1:
            pairs_true_methode2 = pairs_true_methode2 + 1


    number_pred_computed = len(list_true_prediction_pp)
    prediction_success_p_methode2 = count_true_pred_p_methode2
    prediction_success_pp_methode2 = count_true_pred_pp_methode2

    pred_acc_p_methode2 = prediction_success_p_methode2 / number_pred_computed * 100
    pred_acc_pp_methode2 = prediction_success_pp_methode2 / number_pred_computed * 100
    overall_acc_pred_methode2 = (prediction_success_p_methode2 + prediction_success_pp_methode2) / (2 * number_pred_computed) * 100
    Pairs_accuracy_methode2 = pairs_true_methode2 / number_pred_computed * 100

    for index in range(len(list_verb_homog_inf)):
        verb_inf_to_predict = list_verb_homog_inf[index]
        pred_true_p = list_true_prediction_p[index]
        pred_true_pp = list_true_prediction_pp[index]
        results(verb_inf_to_predict, dict_homog_predict_methode2, pred_true_p, pred_true_pp)



#_______________________________________________ Print results for both methods _________________________________________________
    print("------------------------ Acuracy methode 1 -------------------------")
    str_result_methode1 = "Accuracy predicting preterit: ", pred_acc_p_methode1, "% ", "Accuracy predicting past participle: ", pred_acc_pp_methode1, "% ", "Total accurancy: ", str(
        overall_acc_pred_methode1), "%" + "Pairs corectly predicted :", Pairs_accuracy_methode1, "%"
    list_results.append(str_result_methode1)
    list_acc_p_methode1.append(pred_acc_p_methode1)
    list_acc_pp_methode1.append(pred_acc_pp_methode1)
    print(str_result_methode1)

    print("------------------------ Acuracy methode 2 -------------------------")
    str_result_methode2 = "Accuracy predicting preterit: ", pred_acc_p_methode2, "% ", "Accuracy predicting past participle: ", pred_acc_pp_methode2, "% ", "Total accurancy: ", str(overall_acc_pred_methode2), "%" + "Pairs corectly predicted :", Pairs_accuracy_methode2, "%"
    list_results.append(str_result_methode2)
    list_acc_p_methode2.append(pred_acc_p_methode2)
    list_acc_pp_methode2.append(pred_acc_pp_methode2)
    print(str_result_methode2)


    print("Iteration: " + str(iter_w2v))

2022-01-10 17:13:20,358 : INFO : collecting all words and their counts
2022-01-10 17:13:20,360 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-01-10 17:13:20,439 : INFO : PROGRESS: at sentence #10000, processed 153991 words, keeping 19056 word types
2022-01-10 17:13:20,523 : INFO : PROGRESS: at sentence #20000, processed 322539 words, keeping 31880 word types
2022-01-10 17:13:20,645 : INFO : PROGRESS: at sentence #30000, processed 492558 words, keeping 41164 word types
2022-01-10 17:13:20,762 : INFO : PROGRESS: at sentence #40000, processed 689493 words, keeping 48257 word types
2022-01-10 17:13:20,886 : INFO : PROGRESS: at sentence #50000, processed 939605 words, keeping 56902 word types
2022-01-10 17:13:20,952 : INFO : PROGRESS: at sentence #60000, processed 1129858 words, keeping 61691 word types
2022-01-10 17:13:21,067 : INFO : PROGRESS: at sentence #70000, processed 1362820 words, keeping 68056 word types
2022-01-10 17:13:21,191 : INFO : PROGRESS: a

2022-01-10 17:13:51,908 : INFO : EPOCH 3 - PROGRESS: at 19.51% examples, 364281 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:13:52,930 : INFO : EPOCH 3 - PROGRESS: at 27.55% examples, 368357 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:13:53,942 : INFO : EPOCH 3 - PROGRESS: at 36.95% examples, 369449 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:13:54,945 : INFO : EPOCH 3 - PROGRESS: at 45.68% examples, 370782 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:13:55,954 : INFO : EPOCH 3 - PROGRESS: at 55.88% examples, 371519 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:13:56,962 : INFO : EPOCH 3 - PROGRESS: at 63.99% examples, 370847 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:13:57,967 : INFO : EPOCH 3 - PROGRESS: at 72.99% examples, 370338 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:13:58,973 : INFO : EPOCH 3 - PROGRESS: at 81.73% examples, 370383 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:13:59,975 : INFO : EPOCH 3 - PROGRESS: at 89.94% examples, 369352 words/s, in_qsiz

2022-01-10 17:14:51,850 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-01-10 17:14:51,851 : INFO : EPOCH - 7 : training on 6551417 raw words (4167883 effective words) took 12.6s, 330176 effective words/s
2022-01-10 17:14:52,873 : INFO : EPOCH 8 - PROGRESS: at 9.76% examples, 321370 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:14:53,891 : INFO : EPOCH 8 - PROGRESS: at 17.22% examples, 328853 words/s, in_qsize 6, out_qsize 0
2022-01-10 17:14:54,892 : INFO : EPOCH 8 - PROGRESS: at 23.61% examples, 314443 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:14:55,896 : INFO : EPOCH 8 - PROGRESS: at 28.59% examples, 292784 words/s, in_qsize 6, out_qsize 0
2022-01-10 17:14:56,914 : INFO : EPOCH 8 - PROGRESS: at 35.00% examples, 281972 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:14:57,926 : INFO : EPOCH 8 - PROGRESS: at 40.88% examples, 277696 words/s, in_qsize 5, out_qsize 0
2022-01-10 17:14:58,941 : INFO : EPOCH 8 - PROGRESS: at 45.43% examples, 264408 words/s, in_qs

------------------------ Acuracy methode 1 -------------------------
('Accuracy predicting preterit: ', 74.67532467532467, '% ', 'Accuracy predicting past participle: ', 67.53246753246754, '% ', 'Total accurancy: ', '71.1038961038961', '%Pairs corectly predicted :', 56.493506493506494, '%')
------------------------ Acuracy methode 2 -------------------------
('Accuracy predicting preterit: ', 66.88311688311688, '% ', 'Accuracy predicting past participle: ', 63.63636363636363, '% ', 'Total accurancy: ', '65.25974025974025', '%Pairs corectly predicted :', 55.1948051948052, '%')
Iteration: 0


2022-01-10 17:16:57,827 : INFO : PROGRESS: at sentence #30000, processed 492558 words, keeping 41164 word types
2022-01-10 17:16:57,928 : INFO : PROGRESS: at sentence #40000, processed 689493 words, keeping 48257 word types
2022-01-10 17:16:58,014 : INFO : PROGRESS: at sentence #50000, processed 939605 words, keeping 56902 word types
2022-01-10 17:16:58,077 : INFO : PROGRESS: at sentence #60000, processed 1129858 words, keeping 61691 word types
2022-01-10 17:16:58,172 : INFO : PROGRESS: at sentence #70000, processed 1362820 words, keeping 68056 word types
2022-01-10 17:16:58,248 : INFO : PROGRESS: at sentence #80000, processed 1587873 words, keeping 74031 word types
2022-01-10 17:16:58,318 : INFO : PROGRESS: at sentence #90000, processed 1794865 words, keeping 77445 word types
2022-01-10 17:16:58,385 : INFO : PROGRESS: at sentence #100000, processed 1991089 words, keeping 83239 word types
2022-01-10 17:16:58,471 : INFO : PROGRESS: at sentence #110000, processed 2188037 words, keeping 8

...

------------------------ Acuracy methode 1 -------------------------
('Accuracy predicting preterit: ', 78.57142857142857, '% ', 'Accuracy predicting past participle: ', 66.23376623376623, '% ', 'Total accurancy: ', '72.40259740259741', '%Pairs corectly predicted :', 57.14285714285714, '%')
------------------------ Acuracy methode 2 -------------------------
('Accuracy predicting preterit: ', 68.83116883116884, '% ', 'Accuracy predicting past participle: ', 64.93506493506493, '% ', 'Total accurancy: ', '66.88311688311688', '%Pairs corectly predicted :', 56.493506493506494, '%')
Iteration: 38


2022-01-10 20:21:03,797 : INFO : PROGRESS: at sentence #40000, processed 689493 words, keeping 48257 word types
2022-01-10 20:21:03,907 : INFO : PROGRESS: at sentence #50000, processed 939605 words, keeping 56902 word types
2022-01-10 20:21:03,983 : INFO : PROGRESS: at sentence #60000, processed 1129858 words, keeping 61691 word types
2022-01-10 20:21:04,065 : INFO : PROGRESS: at sentence #70000, processed 1362820 words, keeping 68056 word types
2022-01-10 20:21:04,152 : INFO : PROGRESS: at sentence #80000, processed 1587873 words, keeping 74031 word types
2022-01-10 20:21:04,209 : INFO : PROGRESS: at sentence #90000, processed 1794865 words, keeping 77445 word types
2022-01-10 20:21:04,310 : INFO : PROGRESS: at sentence #100000, processed 1991089 words, keeping 83239 word types
2022-01-10 20:21:04,363 : INFO : PROGRESS: at sentence #110000, processed 2188037 words, keeping 89568 word types
2022-01-10 20:21:04,444 : INFO : PROGRESS: at sentence #120000, processed 2380889 words, keeping

2022-01-10 20:21:39,539 : INFO : EPOCH 3 - PROGRESS: at 57.92% examples, 330219 words/s, in_qsize 6, out_qsize 0
2022-01-10 20:21:40,541 : INFO : EPOCH 3 - PROGRESS: at 65.57% examples, 332261 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:21:41,547 : INFO : EPOCH 3 - PROGRESS: at 74.16% examples, 334431 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:21:42,548 : INFO : EPOCH 3 - PROGRESS: at 82.18% examples, 335023 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:21:43,551 : INFO : EPOCH 3 - PROGRESS: at 90.26% examples, 336628 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:21:44,560 : INFO : EPOCH 3 - PROGRESS: at 98.57% examples, 337650 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:21:44,736 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-01-10 20:21:44,764 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-01-10 20:21:44,777 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-01-10 20:21:44,779 : INFO : EPOCH - 3 : trai

2022-01-10 20:22:37,423 : INFO : EPOCH 8 - PROGRESS: at 25.41% examples, 342790 words/s, in_qsize 6, out_qsize 0
2022-01-10 20:22:38,448 : INFO : EPOCH 8 - PROGRESS: at 33.95% examples, 343077 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:22:39,483 : INFO : EPOCH 8 - PROGRESS: at 42.74% examples, 343487 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:22:40,488 : INFO : EPOCH 8 - PROGRESS: at 52.33% examples, 345614 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:22:41,492 : INFO : EPOCH 8 - PROGRESS: at 60.00% examples, 345555 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:22:42,506 : INFO : EPOCH 8 - PROGRESS: at 69.03% examples, 345868 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:22:43,507 : INFO : EPOCH 8 - PROGRESS: at 76.67% examples, 346530 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:22:44,508 : INFO : EPOCH 8 - PROGRESS: at 84.63% examples, 345988 words/s, in_qsize 5, out_qsize 0
2022-01-10 20:22:45,521 : INFO : EPOCH 8 - PROGRESS: at 93.01% examples, 346500 words/s, in_qsiz

------------------------ Acuracy methode 1 -------------------------
('Accuracy predicting preterit: ', 75.97402597402598, '% ', 'Accuracy predicting past participle: ', 69.48051948051948, '% ', 'Total accurancy: ', '72.72727272727273', '%Pairs corectly predicted :', 58.44155844155844, '%')
------------------------ Acuracy methode 2 -------------------------
('Accuracy predicting preterit: ', 66.88311688311688, '% ', 'Accuracy predicting past participle: ', 63.63636363636363, '% ', 'Total accurancy: ', '65.25974025974025', '%Pairs corectly predicted :', 53.896103896103895, '%')
Iteration: 39


In [16]:
# ____________________________ average true prediction per frequency range for all iterations _________________________________


#Ici possibilité de modifier les plages de fréquences retenues pour calculer la précision des prédictions de l'algorithme par tranche de fréquence
#instruction: print les verbes par tranche de fréquence, le nombre de prédiction exacte pour chaque verbe, la précision moyenne des prédictions par tranche de fréquence.

#les paramêtres de la fonction que vous pouvez modifier: (start_value, stop_value, step_value, dict_occ_tupple, dict_predict, form, print_true)

#start_value: valeur minimale de l'itération par tranche de fréquence
#stop_value: valeur maximale de l'itération par tranche de fréquence
#step_value: taille de la tranche de fréquence 
#ex: start_value = 10, stop_value = 25, step_value = 5: l'algorithme vous retourne les tranches de fréquence: 11-15, 16-20, 21-25 
#(la borne minimale de l'intervalle de fréquence est exclue sinon un verbe pourrait se trouver dans 2 tranches de fréquences simultanément)

#dict_occ_tupple, dict_predict, ces deux paramêtres dépendent de la méthode que vous voulez tester, 
#ils ne sont pas à modifier dans la mesure où les 2 méthodes sont déjà présentes

#form: choisir sur quelle forme est calculée la fréquence minimale (par exemple les fréquences minimales pour les formes prétérit form="p"
#les fréquences minimales parmi prétérit et participe passé confondues form="p_pp" ect...) Paramètres possibles: "p", "pp", "inf", "p_pp", "inf_p_pp"

#print_true: ce paramêtre est une bidouille dégueulasse pour print la moyenne sur l'ensemble tranches sans re-print tous les verbes. 
#Dégueulasse mais fontionnel. Je sais comment améliorer ça mais pas le temps + flemme. 

In [17]:
print("----------------- accuracy per frequency methode 1 ----------------------------")
compute_accuracy_frequency(50, 200, 50, dict_homog_occ_tupple, dict_homog_predict_methode1, "inf_p_pp", "True")
compute_accuracy_frequency(200, 500, 100, dict_homog_occ_tupple, dict_homog_predict_methode1,"inf_p_pp", "True")
compute_accuracy_frequency(500, 900, 200, dict_homog_occ_tupple, dict_homog_predict_methode1, "inf_p_pp", "True")
compute_accuracy_frequency(900, 1900, 222222, dict_homog_occ_tupple, dict_homog_predict_methode1, "inf_p_pp", "True")
compute_accuracy_frequency(0, 11000, 11111, dict_homog_occ_tupple, dict_homog_predict_methode1, "inf_p_pp", "False")

----------------- accuracy per frequency methode 1 ----------------------------
______________________ 50  < Min occurence <  100 ____________________________
Numbers of true pairs predicted for  like :  0
Numbers of true pairs predicted for  apply :  38
Numbers of true pairs predicted for  emerge :  29
Numbers of true pairs predicted for  gain :  18
Numbers of true pairs predicted for  seek :  40
Numbers of true pairs predicted for  approach :  2
Numbers of true pairs predicted for  love :  38
Numbers of true pairs predicted for  recognise :  10
Numbers of true pairs predicted for  miss :  0
Numbers of true pairs predicted for  acquire :  27
Numbers of true pairs predicted for  observe :  1
Numbers of true pairs predicted for  accept :  35
Numbers of true pairs predicted for  earn :  0
Numbers of true pairs predicted for  demonstrate :  40
Numbers of true pairs predicted for  arrange :  40
Numbers of true pairs predicted for  collect :  37
Numbers of true pairs predicted for  succeed 

In [18]:
print("----------------- accuracy per frequency methode 2 ----------------------------")
compute_accuracy_frequency(50, 200, 50, dict_homog_occ_tupple, dict_homog_predict_methode2, "p_pp", "True")
compute_accuracy_frequency(200, 500, 100, dict_homog_occ_tupple, dict_homog_predict_methode2,"p_pp", "True")
compute_accuracy_frequency(500, 700, 200, dict_homog_occ_tupple, dict_homog_predict_methode2, "p_pp", "True")
compute_accuracy_frequency(700, 1200, 500, dict_homog_occ_tupple, dict_homog_predict_methode2, "p_pp", "True")
compute_accuracy_frequency(1200, 10000, 222222, dict_homog_occ_tupple, dict_homog_predict_methode2, "p_pp", "True")
compute_accuracy_frequency(0, 11000, 16000, dict_homog_occ_tupple, dict_homog_predict_methode2, "p_pp", "False")

----------------- accuracy per frequency methode 2 ----------------------------
______________________ 50  < Min occurence <  100 ____________________________
Numbers of true pairs predicted for  like :  1
Numbers of true pairs predicted for  apply :  20
Numbers of true pairs predicted for  emerge :  26
Numbers of true pairs predicted for  gain :  0
Numbers of true pairs predicted for  seek :  36
Numbers of true pairs predicted for  approach :  0
Numbers of true pairs predicted for  love :  35
Numbers of true pairs predicted for  recognise :  1
Numbers of true pairs predicted for  miss :  33
Numbers of true pairs predicted for  acquire :  16
Numbers of true pairs predicted for  observe :  4
Numbers of true pairs predicted for  accept :  30
Numbers of true pairs predicted for  earn :  25
Numbers of true pairs predicted for  demonstrate :  40
Numbers of true pairs predicted for  arrange :  8
Numbers of true pairs predicted for  collect :  27
Numbers of true pairs predicted for  declare :